In [ ]:
start_time_l = [i * test_config.stagger_start_time_dt for i in range(test_config.num_agents)]

In [2]:
import os
import torch
from torch_robotics.torch_utils.seed import fix_random_seed
from torch_robotics.torch_utils.torch_utils import get_torch_device
from mmd.trainer import get_dataset
from mmd.utils.loading import load_params_from_yaml


seed = 18 #params.seed
fix_random_seed(seed)

TRAINED_MODELS_DIR = '../../data_trained_models/'
trained_models_dir = TRAINED_MODELS_DIR
model_id = 'EnvEmptyNoWait2D-RobotPlanarDisk'
model_dir = os.path.join(trained_models_dir, model_id)
results_dir = os.path.join(model_dir, 'results_inference', str(seed))
os.makedirs(results_dir, exist_ok=True)
args = load_params_from_yaml(os.path.join(model_dir, "args.yaml"))

device='cuda' #params.device
device = get_torch_device(device)
tensor_args = {'device': device, 'dtype': torch.float32}

train_subset, train_dataloader, val_subset, val_dataloader = get_dataset(
    dataset_class='TrajectoryDataset',
    use_extra_objects=True,
    obstacle_cutoff_margin=0.05,
    **args,
    tensor_args=tensor_args
)
dataset = train_subset.dataset



---------------Loading data
Precomputing the SDF grid and gradients took: 0.305 sec


/local-scratch/localhome/lya108/lufan/mmd/mmd/datasets/trajectories.py:91: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  trajs_free_tmp = torch.load(


TrajectoryDataset
n_trajs: 10000
trajectory_dim: (64, 4)



In [3]:
dir(dataset)

['__abstractmethods__',
 '__add__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_is_protocol',
 'args',
 'base_dir',
 'dataset_shape',
 'dataset_subdir',
 'env',
 'field_key_task',
 'field_key_traj',
 'fields',
 'get_hard_conditions',
 'get_single_pt_hard_conditions',
 'get_unnormalized',
 'include_velocity',
 'load_trajectories',
 'map_task_id_to_trajectories_id',
 'map_trajectory_id_to_task_id',
 'metadata',
 'n_support_points',
 'n_trajs',
 'normalize',
 'normalize_all_data',
 'normalize_tasks',
 'normalize_trajectories',
 'normalizer',
 'normaliz

In [26]:
a, b, c = torch.randn((2,3,4,5)), torch.randn((2,3,4,5)), torch.randn((2,3,4,5))

In [27]:
x = [a]

In [29]:
x.append(b)
x.append(c)

In [30]:
y = torch.stack(x, dim=0)

In [31]:
y.shape

torch.Size([4, 2, 3, 4, 5])